In [40]:
import mlflow

mlflow.set_tracking_uri('http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000')

In [41]:
mlflow.set_experiment('Exp 2 - BoW vs TfIdf')

<Experiment: artifact_location='s3://mlflow-bucket-vietnq/596004659011681108', creation_time=1754205017266, experiment_id='596004659011681108', last_update_time=1754205017266, lifecycle_stage='active', name='Exp 2 - BoW vs TfIdf', tags={}>

In [42]:
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [43]:
df = pd.read_csv('dataset.csv')
df.dropna(inplace=True)
df.shape

(36662, 3)

In [44]:
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    if vectorizer_type == 'BoW':
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

ngram_ranges = [(1, 1), (1, 2), (1, 3)]
max_features = 5000
for ngram_range in ngram_ranges:
    run_experiment("BoW", ngram_range=ngram_range, vectorizer_max_features=max_features, vectorizer_name="BoW")
    run_experiment("TfIdf", ngram_range=ngram_range, vectorizer_max_features=max_features, vectorizer_name="TfIdf")

2025/08/03 14:38:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 14:38:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/596004659011681108/runs/e8a9f7ad3eec4849995c43f4321009f9
🧪 View experiment at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/596004659011681108


2025/08/03 14:38:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 14:39:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TfIdf_(1, 1)_RandomForest at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/596004659011681108/runs/8192fc30bf4f4c6a8f41022d4b3eefe1
🧪 View experiment at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/596004659011681108


2025/08/03 14:39:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 14:39:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TfIdf_(1, 2)_RandomForest at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/596004659011681108/runs/a260ba4a8fde4d7bb6d00f6c54b2f65e
🧪 View experiment at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/596004659011681108


2025/08/03 14:40:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 14:40:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/596004659011681108/runs/c6c70803e3c34c2aa2a106dd07d8805c
🧪 View experiment at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/596004659011681108


2025/08/03 14:41:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 14:41:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TfIdf_(1, 3)_RandomForest at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/596004659011681108/runs/70b819eedca54670b8e0c87644ff60c7
🧪 View experiment at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/596004659011681108


In [47]:
df.to_csv('dataset.csv')